<a href="https://colab.research.google.com/github/RimAmarat/MNIST_DNN/blob/main/MNIST_NN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torchvision import datasets, transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
from tabulate import tabulate

#### Loading the data

In [ ]:
training = datasets.MNIST(root = './data',train=True,download=True,
                         transform = transforms.Compose([transforms.ToTensor()]))

testing = datasets.MNIST(root = './data',train=False,download=True,
                         transform = transforms.Compose([transforms.ToTensor()]))

# Getting the training and test set as tensors for our neural network

train_set = torch.utils.data.DataLoader(training,
                                         batch_size=10,
                                        shuffle=True)
test_set = torch.utils.data.DataLoader(testing,
                                         batch_size=10,
                                        shuffle=True)

100%|██████████| 9.91M/9.91M [00:00<00:00, 17.8MB/s]
100%|██████████| 28.9k/28.9k [00:00<00:00, 493kB/s]
100%|██████████| 1.65M/1.65M [00:00<00:00, 3.89MB/s]
100%|██████████| 4.54k/4.54k [00:00<00:00, 5.06MB/s]


In [ ]:
training.head()

AttributeError: 'MNIST' object has no attribute 'head'

#### Creating the neural network
2 hidden layers of 64 nodes

In [ ]:
class Network(nn.Module):
    def __init__(self):
        super().__init__()
        # 28 x 28 = 784
        self.input_layer = nn.Linear(784,64)
        self.hidden1 = nn.Linear(64,64)
        self.hidden2 = nn.Linear(64,64)
        self.output_layer = nn.Linear(64,10)

    def forward(self, data):
        data = F.relu(self.input_layer(data))
        data = F.relu(self.hidden1(data))
        data = F.relu(self.hidden2(data))
        data = self.output_layer(data)


        return F.log_softmax(data, dim=1)

In [ ]:
network = Network()

learn_rate = optim.Adam(network.parameters(), lr=0.009)
epochs = 10

In [ ]:
loss_table = [['Iteration','Loss']]
for i in range(epochs):
    for data in train_set:
        image, output = data
        network.zero_grad()
        # Resetting the netword's gradient
        # with that we ignore all the loss done previously
        result = network(image.view(-1,784))
        loss = F.nll_loss(result, output)
        loss.backward()
        learn_rate.step()
    loss_table.append([i,loss.detach().numpy()])
    print("iteration",i,"->",loss.detach().numpy())

print(tabulate(loss_table, headers="firstrow", tablefmt="fancy_grid"))

iteration 0 -> 1.7509329
iteration 1 -> 0.27962953
iteration 2 -> 0.05526579
iteration 3 -> 0.1307676
iteration 4 -> 0.08249007
iteration 5 -> 0.0059367204
iteration 6 -> 0.01887448
iteration 7 -> 0.0021791006
iteration 8 -> 0.035174157
iteration 9 -> 0.00023970744
╒═════════════╤═════════════╕
│   Iteration │        Loss │
╞═════════════╪═════════════╡
│           0 │ 1.75093     │
├─────────────┼─────────────┤
│           1 │ 0.27963     │
├─────────────┼─────────────┤
│           2 │ 0.0552658   │
├─────────────┼─────────────┤
│           3 │ 0.130768    │
├─────────────┼─────────────┤
│           4 │ 0.0824901   │
├─────────────┼─────────────┤
│           5 │ 0.00593672  │
├─────────────┼─────────────┤
│           6 │ 0.0188745   │
├─────────────┼─────────────┤
│           7 │ 0.0021791   │
├─────────────┼─────────────┤
│           8 │ 0.0351742   │
├─────────────┼─────────────┤
│           9 │ 0.000239707 │
╘═════════════╧═════════════╛


#### Evaluation

In [ ]:
network.eval()
correct_pred = 0
incorrect_pred = 0
total = 0
with torch.no_grad():
    # keeps us from using backprop
    # speeds up computation
    for data in test_set:
        image, output = data
        result = network(image.view(-1,784))
        for index,tensor_val in enumerate(result):
            total+=1
            if torch.argmax(tensor_val) == output[index]:
                correct_pred+=1
            else:
                incorrect_pred+=1

accuracy = correct_pred / total
print("accuracy ",accuracy)

accuracy  0.9536
